In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

plotly.__version__

'5.14.1'

## 1. Data Populasi dan Atribut Demografi Indonesia

In [5]:
data_pop = pd.read_csv("raw_data/data_penduduk/population_worldwide.csv")
data_growth_pop = pd.read_csv("raw_data/data_penduduk/growth_population.csv").iloc[:, :-1]
indo_stats = pd.read_csv("raw_data/data_penduduk/statistik_indo.csv")

In [6]:
tr_indo_stats = indo_stats.iloc[:, 2:-1].T.drop(["Indicator Name", "Indicator Code"]).iloc[40:, :]
tr_indo_stats.columns = indo_stats["Indicator Name"].values
tr_indo_stats = tr_indo_stats.reset_index()

In [7]:
def transform_pop_data(df: pd.DataFrame, countries: str, value_col_name):
    new_df = pd.DataFrame()
    for country in countries:
        population_df = df[df["Country Name"] == country].iloc[:, 4:]
        reformed_pop = {"negara":[country for _ in population_df.columns],
                        "tahun":[tahun for tahun in population_df.columns], 
                        value_col_name:[population_df[tahun].values[0] for tahun in population_df.columns]}
        new_df = pd.concat([new_df, pd.DataFrame(reformed_pop)])
    return new_df

In [53]:
indo_population = transform_pop_data(data_pop, countries=["Indonesia"], value_col_name="total_penduduk").iloc[40:, :].reset_index(drop=True)

data_nasional = pd.merge(indo_population, tr_indo_stats, left_on="tahun", right_on="index").drop(["index"], axis=1)
data_nasional = data_nasional.dropna(axis=1)

# for col in data_nasional.columns:
#     data_nasional[col] = data_nasional[col].astype(float)
lst_2022 = ["Indonesia", "2022", 275361267.0] + [np.nan for _ in range(len(data_nasional.columns)-3)]
data_nasional.loc[len(data_nasional)] = lst_2022
data_nasional.head()

,negara,tahun,total_penduduk,Perdagangan barang (% dari PDB),"Populasi, total",Pertumbuhan populasi (% tahunan),"Tingkat kesuburan, total (kelahiran per wanita).","Harapan hidup saat lahir, total (tahun)","Angka kematian, bayi (per 1.000 kelahiran hidup)",Migrasi bersih,"Pengangguran, total (% dari angkatan kerja total)",Rasio jumlah masyarakat miskin pada garis kemiskinan nasional (% dari populasi),Indeks GINI,"Rasio jumlah masyarakat miskin pada $1,90 per hari (PPP) (% dari populasi)",Distribusi inkam yang dipegang oleh 20% terendah,"Imunisasi, campak (% dari anak usia 12-23 bulan)","Angka kematian, balita (per 1.000 kelahiran hidup)","Prevalensi HIV, total (% dari populasi usia 15-49)","INB per kapita, PPP (mata uang $ internasional)","INB per kapita, metode Atlas (mata uang US$)","INB, PPP (mata uang $ internasional)","INB, metode Atlas (mata uang US$)","PDB per kapita, PPP (mata uang $ internasional)",PDB per kapita (mata uang US$),Pertumbuhan PDB (% tahunan),PDB (mata uang US$),"Inflasi, deflator PDB (% tahunan)","Industri, nilai tambah (% dari PDB)","Pertanian, nilai tambah (% dari PDB)",Impor barang dan jasa (% dari PDB),Formasi modal bruto (% dari PDB),Ekspor barang dan jasa (% dari PDB),Pengeluaran militer (% dari PDB),Langganan telepon seluler (per 100 orang),"Inflasi, harga konsumen (% tahunan)","Jasa utang total (% dari ekspor barang, jasa dan pendapatan utama)",Bantuan pembangunan resmi bersih yang diterima (mata uang US$),"Stok utang luar negeri, total (DOD, mata uang US$)","Penarikan pribadi, diterima (mata uang US$)","Investasi langsung asing, arus masuk bersih (BoP, mata uang US$)"
0,Indonesia,2000,214072421.0,66.050983,214072421.0,1.447088,2.536,66.433,41.0,-42697.0,6.08,19.1,29.5,43.6,9.4,76.0,52.2,0.1,4370.0,570.0,935805000000.0,122450000000.0,4681.91945,770.865352,4.920068,165021000000.0,20.447457,41.969075,15.678704,30.459567,22.245697,40.977308,0.684484,1.714059,3.688619,22.847663,1645430054.0,144049000000.0,1190200000.0,-4550355286.0
1,Indonesia,2001,217112437.0,59.14416,217112437.0,1.410099,2.497,66.758,39.4,-64261.0,6.08,18.4,30.0,39.9,9.4,76.0,49.9,0.1,4710.0,710.0,1022800000000.0,154215000000.0,4892.352515,739.003946,3.643466,160447000000.0,14.295715,47.893501,15.994918,30.761068,22.539266,39.032139,0.572793,3.003488,11.500115,23.958043,1458180054.0,132710000000.0,1046000000.0,-2977391857.0
2,Indonesia,2002,220115092.0,49.834251,220115092.0,1.373519,2.456,67.134,37.8,-83985.0,6.6,18.2,32.8,26.8,8.8,72.0,47.6,0.1,4970.0,780.0,1093550000000.0,172499000000.0,5121.33454,888.90139,4.499475,195661000000.0,5.896052,47.745331,16.319669,26.391841,21.40407,32.687621,0.70012,5.315401,11.900118,25.22533,1296010010.0,128444000000.0,1258917563.0,145085548.7
3,Indonesia,2003,223080121.0,45.279586,223080121.0,1.338044,2.428,67.413,36.4,-97733.0,6.66,17.4,33.0,26.4,8.6,74.0,45.5,0.1,5190.0,890.0,1158180000000.0,198234000000.0,5399.32871,1052.413199,4.780369,234772000000.0,5.487429,43.749566,15.185348,23.138839,25.598498,30.477655,0.909285,8.290856,6.757317,26.323747,1738630005.0,134373000000.0,1488709351.0,-596923827.8
4,Indonesia,2004,225938595.0,48.919767,225938595.0,1.273226,2.418,65.75,37.1,-102669.0,7.3,16.7,33.9,27.0,8.2,76.0,49.6,0.1,5490.0,1070.0,1239430000000.0,241393000000.0,5749.521451,1136.755211,5.030874,256837000000.0,8.550727,44.627616,14.33578,27.5446,24.056366,32.216695,0.945715,13.426926,6.06406,25.467404,97559997.56,138042000000.0,1866314990.0,1896082770.0


In [46]:
rank_pop = transform_pop_data(data_pop, countries=["Indonesia", "Pakistan", "United States", "China", "India"], value_col_name="total_penduduk")
growth_pop = transform_pop_data(data_growth_pop, countries=["Indonesia", "Pakistan", "United States", "China", "India"], value_col_name="pertumbuhan_penduduk").dropna()

rank_pop_21 = rank_pop[rank_pop["tahun"] == "2021"].reset_index(drop=True)

In [70]:
data_nasional.to_csv("preprocessed_data/data_nasional.csv", index=False)
rank_pop_21.to_csv("preprocessed_data/peringkat_total_penduduk_dunia.csv", index=False)
growth_pop.to_csv("preprocessed_data/data_pertumbuhan_penduduk.csv", index=False)

## 2. Data Distribusi Umur Nasional

In [47]:
data_umur_0910 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2009-2010.xlsx")
data_umur_1112 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2011-2012.xlsx")
data_umur_1314 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2013-2014.xlsx")
data_umur_1516 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2015-2016.xlsx")
data_umur_1718 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2017-2018.xlsx")
data_umur_1920 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2019-2020.xlsx")
data_umur_2122 = pd.read_excel("raw_data\data_umur\Jumlah Penduduk Usia 15 tahun ke Atas Menurut Golongan Umur 2021-2022.xlsx")

used_df = [data_umur_0910, data_umur_1112, data_umur_1314, data_umur_1516, data_umur_1718, data_umur_1920, data_umur_2122]

In [48]:
start_year = 2009
dict_umur = {"tahun":[i for i in range(start_year, 2023)], "usia_dibawah_15":[], "usia_produktif":[], "usia_diatas_60":[]}

for df in used_df:
    # print(start_year)
    dict_umur["usia_dibawah_15"].append(data_nasional[data_nasional["tahun"] == str(start_year)]["total_penduduk"].values[0] - int(df.iloc[-1, 1]))
    start_year += 1
    dict_umur["usia_dibawah_15"].append(data_nasional[data_nasional["tahun"] == str(start_year)]["total_penduduk"].values[0] - int(df.iloc[-1, 4]))
    start_year += 1

    dict_umur["usia_produktif"].append(sum(df.iloc[2:-2, 1].astype(int).values))
    dict_umur["usia_produktif"].append(sum(df.iloc[2:-2, 4].astype(int).values))

    dict_umur["usia_diatas_60"].append(int(df.iloc[-2, 1]))
    dict_umur["usia_diatas_60"].append(int(df.iloc[-2, 4]))

In [49]:
df_umur = pd.DataFrame(dict_umur)
df_umur["total"] = df_umur["usia_dibawah_15"] + df_umur["usia_produktif"] + df_umur["usia_diatas_60"]
df_umur["dependency_ratio"] = (df_umur["usia_dibawah_15"] + (df_umur["usia_diatas_60"] * 0.6)) / df_umur["usia_produktif"]

In [50]:
# dict_umur = {"tahun":[2018, 2019, 2020, 2021, 2022], "usia_dibawah_15":[], "usia_produktif":[], "usia_diatas_60":[]}

# for df in used_df:
#     for col in df.iloc[:, 1:].columns:
#         if df[col].dtypes == df[data_umur_2018.columns[0]].dtypes: 
#             df[col] = df[col].apply(lambda x: str(x).replace(' ','').replace(',','.'))
#             df[col] = df[col].astype(float)

#     dict_umur["usia_dibawah_15"].append(sum(df.iloc[:3, -1:].values)[0])
#     dict_umur["usia_produktif"].append(sum(df.iloc[3:-4, -1:].values)[0])
#     dict_umur["usia_diatas_60"].append(sum(df.iloc[-4:, -1:].values)[0])

# umur_df = pd.DataFrame(dict_umur)
# for col in umur_df.columns:
#     umur_df[col] = umur_df[col].astype(int)

# umur_df["total"] = umur_df["usia_dibawah_15"] + umur_df["usia_produktif"] + umur_df["usia_diatas_60"]
# umur_df["persentase_usia_dibawah_15"] = umur_df["usia_dibawah_15"] / umur_df["total"]
# umur_df["persentase_usia_produktif"] = umur_df["usia_produktif"] / umur_df["total"]
# umur_df["persentase_usia_diatas_60"] = umur_df["usia_diatas_60"] / umur_df["total"]

In [16]:
df_umur.to_csv("preprocessed_data/data_klasifikasi_umur2.csv", index=False)

## 3. Data penduduk tiap provinsi

In [22]:
penduduk_prov_18_19 = pd.read_excel("raw_data/data_penduduk//Jumlah Penduduk Menurut Provinsi di Indonesia 2018-2019.xlsx").iloc[:-1, :].sort_values(by=["Provinsi di Indonesia"]).reset_index(drop=True)
penduduk_prov_20_22 = pd.read_excel("raw_data/data_penduduk//Jumlah Penduduk Menurut Provinsi di Indonesia 2020-2022.xlsx").iloc[:-1, :].sort_values(by=["Provinsi di Indonesia"]).reset_index(drop=True)

old_prov = ["Kep. Bangka Belitung", "DKI Jakarta", "Kep. Riau", "DI Yogyakarta"]
new_prov = ["Bangka Belitung", "Jakarta Raya", "Riau", "Yogyakarta"]

penduduk_prov = penduduk_prov_18_19.merge(penduduk_prov_20_22, on="Provinsi di Indonesia")

In [23]:
old_prov = ["Kep. Bangka Belitung", "Aceh", "Kep. Riau", "DI Yogyakarta", "Nusa Tenggara Barat"]
new_prov = ["Bangka Belitung", "DI. ACEH", "Riau", "DAERAH ISTIMEWA YOGYAKARTA", "NUSATENGGARA BARAT"]

penduduk_prov["Provinsi di Indonesia"] = penduduk_prov["Provinsi di Indonesia"].replace(old_prov, new_prov)
penduduk_prov["Provinsi di Indonesia"] = penduduk_prov["Provinsi di Indonesia"].apply(lambda x: x.upper())
penduduk_prov.rename(columns={"Provinsi di Indonesia":"Provinsi"}, inplace=True)

In [24]:
penduduk_prov.to_csv("preprocessed_data\penduduk_per_provinsi_processed.csv", index=False)

## 4. Data clustering

In [43]:
data_ekonomi = pd.read_excel("raw_data/data_demografi/[Seri 2010] Produk Domestik Regional Bruto Per Kapita.xlsx")
data_humdev = pd.read_excel("raw_data/data_demografi/Indeks Pembangunan Manusia Menurut Provinsi, 2022.xlsx", decimal=',')
data_kemiskinan = pd.read_excel("raw_data/data_demografi/Jumlah dan Persentase Penduduk Miskin Menurut Provinsi, 2022.xlsx", decimal=',').dropna(axis=1)
data_faskes = pd.read_excel("raw_data/data_demografi/Jumlah Rumah Sakit Umum, Rumah Sakit Khusus, Puskesmas, Klinik Pratama, dan Posyandu Menurut Provinsi, 2020.xlsx")
data_fasped = pd.read_excel("raw_data/data_demografi/Kelurahan yang Memiliki Fasilitas Sekolah Menurut Provinsi, 2021.xlsx")
data_luas = pd.read_excel("raw_data/data_demografi\Luas Daerah dan Jumlah Pulau Menurut Provinsi, 2021.xlsx")

data_ekonomi = data_ekonomi[["Provinsi", "hb_2022", "hk_2022"]]
data_faskes = data_faskes.iloc[:, :-2].replace("...", 0)
data_fasped.columns = ["Provinsi", "kelurahan_jumlah_sd", "kelurahan_jumlah_smp", 
                       "kelurahan_jumlah_sma", "kelurahan_jumlah_smk", "kelurahan_jumlah_pt"]

In [44]:
old_prov_upper = ["ACEH", "KEP. BANGKA BELITUNG", "NUSA TENGGARA BARAT", "DI YOGYAKARTA", "KEP. RIAU"]
old_prov_cap = ["Aceh", "Kepulauan Bangka Belitung", "Nusa Tenggara Barat", "DI Yogyakarta"]

new_prov_geo = ["DI. ACEH", "BANGKA BELITUNG", "NUSATENGGARA BARAT", "DAERAH ISTIMEWA YOGYAKARTA", "KEPULAUAN RIAU"]

data_ekonomi["Provinsi"] = data_ekonomi["Provinsi"].replace(old_prov_upper, new_prov_geo)
data_humdev["Provinsi"] = data_humdev["Provinsi"].replace(old_prov_cap, new_prov_geo[:-1]).apply(lambda x: x.upper())
data_kemiskinan["Provinsi"] = data_kemiskinan["Provinsi"].replace(old_prov_cap, new_prov_geo[:-1]).apply(lambda x: x.upper())
data_faskes["Provinsi"] = data_faskes["Provinsi"].replace(old_prov_cap, new_prov_geo[:-1]).apply(lambda x: x.upper())
data_fasped["Provinsi"] = data_fasped["Provinsi"].replace(old_prov_cap, new_prov_geo[:-1]).apply(lambda x: x.upper())
data_luas["Provinsi"] = data_luas["Provinsi"].replace(old_prov_cap, new_prov_geo[:-1]).apply(lambda x: x.upper())

In [45]:
cluster_df = data_ekonomi.copy()

for df in [data_humdev, data_faskes, data_fasped, 
           data_kemiskinan[["Provinsi", "Persentase Penduduk Miskin - Maret"]],
           data_luas[["Provinsi", "Luas Wilayah (km2)"]], 
           penduduk_prov[["Provinsi", "2022"]]]:
    cluster_df = cluster_df.merge(df, on="Provinsi")

for col in cluster_df.columns:
    if col != "Provinsi" and cluster_df[col].dtypes == cluster_df["Provinsi"].dtypes: 
        cluster_df[col] = cluster_df[col].apply(lambda x: str(x).replace(' ','').replace(',','.'))
    if col != "Provinsi":
        cluster_df[col] = cluster_df[col].astype(float)

cluster_df.to_csv("preprocessed_data/cluster_data.csv", index=False)